In [2]:
!pip3 install bpemb
!pip3 install --force-reinstall nlpaug

     |████████████████████████████████| 1.0MB 5.3MB/s 
     |████████████████████████████████| 92kB 4.7MB/s 


In [0]:
import unicodedata
import string
import re
import random
import time
import datetime
import math
import socket
hostname = socket.gethostname()

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence#, masked_cross_entropy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

In [0]:
from bpemb import BPEmb
import textwrap
import nlpaug

In [5]:
bpemb_en = BPEmb(lang="en", dim=50, vs=1000, add_pad_emb=True)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs1000.model


100%|██████████| 251221/251221 [00:00<00:00, 441631.98B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs1000.d50.w2v.bin.tar.gz


100%|██████████| 190477/190477 [00:00<00:00, 458479.78B/s]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# open text file and read in data as `text`
with open('big.txt', 'r') as f:
    text = f.read()

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
import nlpaug.augmenter.char as nac


In [0]:
def get_encodes(arr, seq_length, use_aug=False):
  if use_aug:
    aug_rr = nac.KeyboardAug(aug_char_min=0, aug_char_max=None, aug_char_p=0.4, aug_word_p=0.4, aug_word_min=0, aug_word_max=arr.size//3, special_char=False)
    
    augmented_data = list(map(lambda x: aug_rr.augment(x), arr.ravel()))
    arr = np.array(augmented_data).reshape(arr.shape)
  flat_arr = arr.ravel()

  def padded_encode(x):
    k = np.full((seq_length,), bpemb_en.vs)
    enc = np.array(bpemb_en.encode_ids(x))
    k[:enc.size] = enc


    return enc.size, k

  res_arr = np.empty((*flat_arr.shape, seq_length), dtype="int32")
  len_vec = np.empty( (arr.shape[0]))
  for i in range(flat_arr.size):

    res = padded_encode(flat_arr[i])  
    res_arr[i] = res[1]
    len_vec[i] = res[0]

  if not use_aug:
    res_arr = np.insert(res_arr, 0, 1, 1)
  len_vec, perm_idx = torch.from_numpy(len_vec).sort(0, descending=True)
  res_arr = res_arr[perm_idx]

  leng, res = len_vec, one_hot_encode(res_arr, bpemb_en.vs + 1)

  leng += (1 if not use_aug else 0)
  leng, res = leng, torch.from_numpy(res)

  return leng, res

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    # total number of batches we can make
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]    

  

    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1]):
        # The features
        base = arr[:, n:n+1]
        # y = np.vectorize(get_int)(base)

        x = base.copy()
        # y = one_hot_encode(y, len(words))
        lengths_x, x = get_encodes(x, seq_length, use_aug=True)
        lengths_y, y = get_encodes(base, seq_length)
        
        yield lengths_x, x, lengths_y, y

In [0]:

arr = np.array(textwrap.wrap(text=text, width=5))

batches = get_batches(arr, 2, 5)

In [13]:
lengths_x, x, lengths_y, y = next(batches)
x.shape

torch.Size([2, 5, 1001])

### Hide


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.gru = nn.GRU(input_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True, batch_first=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        packed = torch.nn.utils.rnn.pack_padded_sequence(input_seqs, input_lengths, batch_first=True)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

In [0]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(1)
        this_batch_size = encoder_outputs.size(0)

        # Create variable to store attention energies
        attn_energies = torch.zeros(this_batch_size, max_len) # B x S


        attn_energies = attn_energies.cuda()

        # For each batch of encoder outputs
        for b in range(this_batch_size):
            # Calculate energy for each encoder output
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[b, :].squeeze(0), encoder_outputs[b, i])

        # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
        return F.softmax(attn_energies).unsqueeze(1)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = hidden.dot(encoder_output)
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.dot(energy)
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = self.v.dot(energy)
            return energy

In [0]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, input_size, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(input_size, hidden_size, n_layers, dropout=dropout, batch_first=True)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # Choose attention model
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):

        rnn_output, hidden = self.gru(input_seq, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs;
        # apply to encoder outputs to get weighted average
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs) # B x S=1 x N

        # Attentional vector using the RNN hidden state and context vector
        # concatenated together (Luong eq. 5)
        rnn_output = rnn_output.squeeze(1) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N

        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = F.tanh(self.concat(concat_input))

        # Finally predict next token (Luong eq. 6, without softmax)
        output = F.log_softmax(self.out(concat_output))

        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

### Imp


In [0]:
def loss(Y_hat, Y):
        # TRICK 3 ********************************
        # before we calculate the negative log likelihood, we need to mask out the activations
        # this means we don't want to take into account padded items in the output vector
        # simplest way to think about this is to flatten ALL sequences into a REALLY long sequence
        # and calculate the loss on that.

        # flatten all the labels
        Y = Y.view(-1)

        # flatten all predictions
        Y_hat = Y_hat.view(-1, Y_hat.size(-1))

        # create a mask by filtering out all tokens that ARE NOT the padding token
        tag_pad_token = 1000
        mask = (Y < tag_pad_token).float()

        # count how many tokens we have
        nb_tokens = int(torch.sum(mask).item())

        # pick the values for the label and zero out the rest with the mask
        Y_hat = Y_hat[range(Y_hat.shape[0]), Y] * mask

        # compute cross entropy loss which ignores all <PAD> tokens
        ce_loss = -torch.sum(Y_hat) / nb_tokens

        return ce_loss
def cus_loss():
  return loss

In [0]:
def train(encoder, decoder, data, epochs=10, batch_size=30, seq_length=500, hidden_size=1000, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    encoder.train()
    decoder.train()
    
    opt1 = torch.optim.Adam(encoder.parameters(), lr=lr)
    opt2 = torch.optim.Adam(decoder.parameters(), lr=lr)

    criterion = cus_loss()


    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    decoder.cuda()
    encoder.cuda()
    
    counter = 0

    for e in range(epochs):
        # initialize hidden state
        h = None

        for lenx, x, leny, y in get_batches(data, batch_size=batch_size, seq_length=seq_length):
          try:
            counter += 1
                        # One-hot encode our data and make them Torch tensors
            lenx, leny = lenx.cuda(), leny.cuda()
            inputs, targets = x.cuda(), y.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = None

            # zero accumulated gradients
            encoder.zero_grad()
            decoder.zero_grad()
            
            max_target_length = int(max(leny).item())
            decoder_input = targets[:, :1]

            all_decoder_outputs = torch.zeros(batch_size, max_target_length, decoder.output_size)

            decoder_input = decoder_input.cuda()
            all_decoder_outputs = all_decoder_outputs.cuda()

            # Run through decoder one time step at a time
            encoder_outputs, encoder_hidden = encoder(inputs, lenx, h)

            decoder_hidden = encoder_hidden[:decoder.n_layers].cuda()

            for t in range(max_target_length):
                decoder_output, decoder_hidden, decoder_attn = decoder(decoder_input, decoder_hidden, encoder_outputs)

                all_decoder_outputs[:, t] = decoder_output
                decoder_input = targets[:, t+1].unsqueeze(1) # Next input is current target
            # get the output from the model
            # h_dec = encoder_outputs
            # print("Encoder_hidden: ", encoder_hidden.size())
            # out, h_dec = decoder(targets[:, :-1], encoder_hidden[:decoder.n_layers], h_dec)
            out = all_decoder_outputs
            targets = targets[:, :out.size(1)]
            tar = targets.argmax(2)
            tar = tar.view(tar.size(0)*tar.size(1))
            cur = out.view(out.size(0)*out.size(1), -1)

            loss = criterion(cur, tar)
            training_loss.append(loss.item())


            # calculate the loss and perform backprop
            # loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(decoder.parameters(), clip)
            nn.utils.clip_grad_norm_(encoder.parameters(), clip)
            opt2.step()
            opt1.step()
            
            
            # loss stats
            if counter % print_every == 0:
                    # Get validation loss
                val_h = None
                val_losses = []
                val_acc = []
                encoder.eval()
                decoder.eval()

                for lenx, x, leny, y in get_batches(val_data, batch_size=batch_size, seq_length=seq_length):

                        # One-hot encode our data and make them Torch tensors
                        
                        # Creating new variables for the hidden state, otherwise
                        # we'd backprop through the entire training history
                    val_h = None
                        
                    inputs, targets = x, y
                    inputs, targets = inputs.cuda(), targets.cuda()
                    lenx, leny = lenx.cuda(), leny.cuda()
                        

                    encoder_outputs, encoder_hidden = encoder(inputs, lenx, val_h)

                    max_target_length = int(max(leny).item())
                    decoder_input = targets[:, :1]

                    all_decoder_outputs = torch.zeros(batch_size, max_target_length, decoder.output_size)

                    decoder_input = decoder_input.cuda()
                    all_decoder_outputs = all_decoder_outputs.cuda()

                    # Run through decoder one time step at a time
                    encoder_outputs, encoder_hidden = encoder(inputs, lenx, h)

                    decoder_hidden = encoder_hidden[:decoder.n_layers].cuda()

                    for t in range(max_target_length - 1):
                        decoder_output, decoder_hidden, decoder_attn = decoder(decoder_input, decoder_hidden, encoder_outputs)

                        all_decoder_outputs[:, t] = decoder_output
                        decoder_input = targets[:, t+1].unsqueeze(1) # Next input is current target
                    
                    # get the output from the model
                    # h_dec = encoder_outputs
                    # print("Encoder_hidden: ", encoder_hidden.size())
                    # out, h_dec = decoder(targets[:, :-1], encoder_hidden[:decoder.n_layers], h_dec)
                    out = all_decoder_outputs
                    targets = targets[:, :out.size(1)]
                    # h_dec = encoder_outputs

                    # out, h_dec = decoder(targets[:, :-1], encoder_hidden[:decoder.n_layers], h_dec)
                    tar = targets.argmax(2)
                    tar = tar.view(tar.size(0)*tar.size(1))
                    cur = out.view(out.size(0)*out.size(1), -1)
                    val_loss = criterion(cur, tar)
              
                    validation_loss.append(val_loss.item())
                    val_losses.append(val_loss.item())
                    output = cur
                    ind = min(len(validation_loss), 10)
                    acc_output = output.cpu().detach()

                    Y = tar.cpu().view(-1)

                    Y_hat = acc_output.argmax(1)
                    Y_hat = Y_hat.view(-1)

                    tag_pad_token = 1000
                    mask = (Y < tag_pad_token).float()

                    nb_tokens = int(torch.sum(mask).item())

                    current_accuracy =  (np.equal(Y_hat.numpy().astype("int32"), Y.numpy()) * mask.numpy()).sum()

                    div = nb_tokens
                    val_acc.append(current_accuracy/(div))
                    validation_accuracy.append(current_accuracy/(div))
                    

                    if min(validation_loss[-ind:]) != min(validation_loss): break
                    
                encoder.train() # reset to train mode after iterationg through validation data
                decoder.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                      "Val Accuracy: {:.4f}".format(np.mean(val_acc)))
          except Exception as ex:
            print(str(e))
            raise e

In [0]:
CLASSES = bpemb_en.vs + 1
hidden_size = 300
decoder = LuongAttnDecoderRNN('general', CLASSES, hidden_size, CLASSES, n_layers=2)

encoder = EncoderRNN(CLASSES, hidden_size, n_layers=2)

sl = 40
data = np.array(textwrap.wrap(text=text, width=sl))

training_loss = []
validation_loss = []
validation_accuracy = []


In [0]:
train(encoder, decoder, data, epochs=10, batch_size=10, seq_length=sl, hidden_size=hidden_size, lr=0.001, clip=5, val_frac=0.1, print_every=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10... Step: 10... Loss: 0.2724... Val Loss: 0.2802 Val Accuracy: 0.9379
Epoch: 1/10... Step: 20... Loss: 0.3358... Val Loss: 0.2982 Val Accuracy: 0.9448
Epoch: 1/10... Step: 30... Loss: 0.3590... Val Loss: 0.3413 Val Accuracy: 0.9241
Epoch: 1/10... Step: 40... Loss: 0.3070... Val Loss: 0.3563 Val Accuracy: 0.9172
Epoch: 1/10... Step: 50... Loss: 0.2360... Val Loss: 0.3612 Val Accuracy: 0.9448
Epoch: 1/10... Step: 60... Loss: 0.2456... Val Loss: 0.3641 Val Accuracy: 0.9379
Epoch: 1/10... Step: 70... Loss: 0.1803... Val Loss: 0.3638 Val Accuracy: 0.9379
Epoch: 1/10... Step: 80... Loss: 0.2267... Val Loss: 0.3477 Val Accuracy: 0.9379
Epoch: 1/10... Step: 90... Loss: 0.2581... Val Loss: 0.3439 Val Accuracy: 0.9448
Epoch: 1/10... Step: 100... Loss: 0.1819... Val Loss: 0.3293 Val Accuracy: 0.9448
Epoch: 1/10... Step: 110... Loss: 0.2353... Val Loss: 0.3378 Val Accuracy: 0.9379
Epoch: 1/10... Step: 120... Loss: 0.2485... Val Loss: 0.3091 Val Accuracy: 0.9379
Epoch: 1/10... Step: 130.

In [0]:
model_name = 'seq2seq_encoder_20_epoch.net'
torch.save(encoder.state_dict(), model_name)


model1_name = 'seq2seq_decoder_20_epoch.net'
torch.save(decoder.state_dict(), model1_name)


In [0]:
def predict(net, word, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        x = np.array([word])
        x = get_encodes(x).reshape(1, 1, -1)
        inputs = torch.from_numpy(x)
        print(inputs.shape)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        return p, h

In [0]:
def sample(encoder, decoder, sentence='The', top_k=None):
    lst_res = []
    encoder.eval()
    decoder.eval() # eval mode
    
        
    h = None
    length, inputs = get_encodes(np.array(sentence[:100]), seq_length=80)
    
    inputs, length = inputs.cuda(), length.cuda()
    res, h = encoder(inputs, length, h)
    while True:
      h_dec = res
      zero_s = np.zeros((100, 1, 10000))
      zero_s[:, :, 0] = 1
      print( h[:decoder.n_layers].shape, h_dec.shape)
      out, h_dec = decoder(zero_s, h[:decoder.n_layers], h_dec)

      tar = inputs.argmax(1)
      tar = tar.view(tar.size(0)*tar.size(1))
      cur = out.view(out.size(0)*out.size(1), -1)
      val_loss = criterion(cur, tar)
      

In [0]:
sample(encoder, decoder, data)